In [1]:
import os
import sys
import argparse
import time
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from conf import settings
from utils import get_network, get_training_dataloader, get_test_dataloader, WarmUpLR, \
    most_recent_folder, most_recent_weights, last_epoch, best_acc_weights

In [12]:
from models.vgg import vgg19_bn
net0= vgg19_bn()
net= "vgg19"
gpu=False
b=128
warm=1
lr=0.1
resume=False
#data preprocessing:
cifar100_training_loader = get_training_dataloader(
    settings.CIFAR100_TRAIN_MEAN,
    settings.CIFAR100_TRAIN_STD,
    num_workers=4,
    batch_size=b,
    shuffle=True
)

cifar100_test_loader = get_test_dataloader(
    settings.CIFAR100_TRAIN_MEAN,
    settings.CIFAR100_TRAIN_STD,
    num_workers=4,
    batch_size=b,
    shuffle=True
)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net0.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=settings.MILESTONES, gamma=0.2) #learning rate decay
iter_per_epoch = len(cifar100_training_loader)
warmup_scheduler = WarmUpLR(optimizer, iter_per_epoch * warm)
writer = SummaryWriter(log_dir=os.path.join(
            settings.LOG_DIR, net, settings.TIME_NOW))
def train(epoch):

    start = time.time()
    net0.train()
    for batch_index, (images, labels) in enumerate(cifar100_training_loader):

        if gpu:
            labels = labels.cuda()
            images = images.cuda()

        optimizer.zero_grad()
        outputs = net0(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        n_iter = (epoch - 1) * len(cifar100_training_loader) + batch_index + 1

        last_layer = list(net0.children())[-1]
        for name, para in last_layer.named_parameters():
            if 'weight' in name:
                writer.add_scalar('LastLayerGradients/grad_norm2_weights', para.grad.norm(), n_iter)
            if 'bias' in name:
                writer.add_scalar('LastLayerGradients/grad_norm2_bias', para.grad.norm(), n_iter)

        print('Training Epoch: {epoch} [{trained_samples}/{total_samples}]\tLoss: {:0.4f}\tLR: {:0.6f}'.format(
            loss.item(),
            optimizer.param_groups[0]['lr'],
            epoch=epoch,
            trained_samples=batch_index * b + len(images),
            total_samples=len(cifar100_training_loader.dataset)
        ))

        #update training loss for each iteration
        writer.add_scalar('Train/loss', loss.item(), n_iter)

        if epoch <= warm:
            warmup_scheduler.step()

    for name, param in net0.named_parameters():
        layer, attr = os.path.splitext(name)
        attr = attr[1:]
        writer.add_histogram("{}/{}".format(layer, attr), param, epoch)

    finish = time.time()

    print('epoch {} training time consumed: {:.2f}s'.format(epoch, finish - start))

@torch.no_grad()
def eval_training(epoch=0, tb=True):

    start = time.time()
    net0.eval()

    test_loss = 0.0 # cost function error
    correct = 0.0

    for (images, labels) in cifar100_test_loader:

        if gpu:
            images = images.cuda()
            labels = labels.cuda()

        outputs = net0(images)
        loss = loss_function(outputs, labels)

        test_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum()

    finish = time.time()
    if gpu:
        print('GPU INFO.....')
        print(torch.cuda.memory_summary(), end='')
    print('Evaluating Network.....')
    print('Test set: Epoch: {}, Average loss: {:.4f}, Accuracy: {:.4f}, Time consumed:{:.2f}s'.format(
        epoch,
        test_loss / len(cifar100_test_loader.dataset),
        correct.float() / len(cifar100_test_loader.dataset),
        finish - start
    ))
    print()

    #add informations to tensorboard
    if tb:
        writer.add_scalar('Test/Average loss', test_loss / len(cifar100_test_loader.dataset), epoch)
        writer.add_scalar('Test/Accuracy', correct.float() / len(cifar100_test_loader.dataset), epoch)

    return correct.float() / len(cifar100_test_loader.dataset)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
if __name__ == '__main__':
    '''
    parser = argparse.ArgumentParser(description='models for classification in cifar100 Training With Pytorch')
    parser.add_argument('--net', type=str,  default="mobilenet", help='net type')  
    #default="vgg19"  default="densenet121" default="resnet101"  default="shufflenet" default="seresnet101"  default="mobilenet"
    parser.add_argument('--gpu', action='store_true', default=False, help='use gpu or not')
    parser.add_argument('--b', type=int, default=128, help='batch size for dataloader')
    parser.add_argument('--warm', type=int, default=1, help='warm up training phase')
    parser.add_argument('--lr', type=float, default=0.1, help='initial learning rate')
    parser.add_argument('--resume', action='store_true', default=False, help='resume training')
    args = parser.parse_args()

    net = get_network(args)
    '''
    from models.vgg import vgg19_bn
    net0= vgg19_bn()
    net= "vgg19"
    gpu=False
    b=128
    warm=1
    lr=0.1
    resume=False
    #data preprocessing:
    cifar100_training_loader = get_training_dataloader(
        settings.CIFAR100_TRAIN_MEAN,
        settings.CIFAR100_TRAIN_STD,
        num_workers=4,
        batch_size=b,
        shuffle=True
    )

    cifar100_test_loader = get_test_dataloader(
        settings.CIFAR100_TRAIN_MEAN,
        settings.CIFAR100_TRAIN_STD,
        num_workers=4,
        batch_size=b,
        shuffle=True
    )

    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net0.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=settings.MILESTONES, gamma=0.2) #learning rate decay
    iter_per_epoch = len(cifar100_training_loader)
    warmup_scheduler = WarmUpLR(optimizer, iter_per_epoch * warm)

    if resume:
        recent_folder = most_recent_folder(os.path.join(settings.CHECKPOINT_PATH, net), fmt=settings.DATE_FORMAT)
        if not recent_folder:
            raise Exception('no recent folder were found')

        checkpoint_path = os.path.join(settings.CHECKPOINT_PATH, net, recent_folder)

    else:
        checkpoint_path = os.path.join(settings.CHECKPOINT_PATH, net, settings.TIME_NOW)

    #use tensorboard
    if not os.path.exists(settings.LOG_DIR):
        os.mkdir(settings.LOG_DIR)

    #since tensorboard can't overwrite old values
    #so the only way is to create a new tensorboard log
    writer = SummaryWriter(log_dir=os.path.join(
            settings.LOG_DIR, net, settings.TIME_NOW))
    input_tensor = torch.Tensor(1, 3, 32, 32)
    if gpu:
        input_tensor = input_tensor.cuda()
    writer.add_graph(net0, input_tensor)

    #create checkpoint folder to save model
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
    checkpoint_path = os.path.join(checkpoint_path, '{net}-{epoch}-{type}.pth')

    best_acc = 0.0
    if resume:
        best_weights = best_acc_weights(os.path.join(settings.CHECKPOINT_PATH, args.net, recent_folder))
        if best_weights:
            weights_path = os.path.join(settings.CHECKPOINT_PATH, net, recent_folder, best_weights)
            print('found best acc weights file:{}'.format(weights_path))
            print('load best training file to test acc...')
            net.load_state_dict(torch.load(weights_path))
            best_acc = eval_training(tb=False)
            print('best acc is {:0.2f}'.format(best_acc))

        recent_weights_file = most_recent_weights(os.path.join(settings.CHECKPOINT_PATH, net, recent_folder))
        if not recent_weights_file:
            raise Exception('no recent weights file were found')
        weights_path = os.path.join(settings.CHECKPOINT_PATH, net, recent_folder, recent_weights_file)
        print('loading weights file {} to resume training.....'.format(weights_path))
        net.load_state_dict(torch.load(weights_path))

        resume_epoch = last_epoch(os.path.join(settings.CHECKPOINT_PATH, net, recent_folder))


    for epoch in range(1, settings.EPOCH + 1):
        if epoch > warm:
            train_scheduler.step(epoch)

        if resume:
            if epoch <= resume_epoch:
                continue

        train(epoch)
        acc = eval_training(epoch)

        #start to save best performance model after learning rate decay to 0.01
        if epoch > settings.MILESTONES[1] and best_acc < acc:
            weights_path = checkpoint_path.format(net=net, epoch=epoch, type='best')
            print('saving weights file to {}'.format(weights_path))
            torch.save(net.state_dict(), weights_path)
            best_acc = acc
            continue

        if not epoch % settings.SAVE_EPOCH:
            weights_path = checkpoint_path.format(net=net, epoch=epoch, type='regular')
            print('saving weights file to {}'.format(weights_path))
            torch.save(net.state_dict(), weights_path)

    writer.close()

Files already downloaded and verified
Files already downloaded and verified
Training Epoch: 1 [128/50000]	Loss: 4.6157	LR: 0.000000
Training Epoch: 1 [256/50000]	Loss: 4.6318	LR: 0.000256
Training Epoch: 1 [384/50000]	Loss: 4.6165	LR: 0.000512
Training Epoch: 1 [512/50000]	Loss: 4.6358	LR: 0.000767
Training Epoch: 1 [640/50000]	Loss: 4.6553	LR: 0.001023
Training Epoch: 1 [768/50000]	Loss: 4.6382	LR: 0.001279
Training Epoch: 1 [896/50000]	Loss: 4.6354	LR: 0.001535
Training Epoch: 1 [1024/50000]	Loss: 4.6401	LR: 0.001790
Training Epoch: 1 [1152/50000]	Loss: 4.6367	LR: 0.002046
Training Epoch: 1 [1280/50000]	Loss: 4.6267	LR: 0.002302
Training Epoch: 1 [1408/50000]	Loss: 4.6241	LR: 0.002558
Training Epoch: 1 [1536/50000]	Loss: 4.6263	LR: 0.002813
Training Epoch: 1 [1664/50000]	Loss: 4.6198	LR: 0.003069
Training Epoch: 1 [1792/50000]	Loss: 4.6111	LR: 0.003325
Training Epoch: 1 [1920/50000]	Loss: 4.6626	LR: 0.003581
Training Epoch: 1 [2048/50000]	Loss: 4.6186	LR: 0.003836
Training Epoch: 1 [

KeyboardInterrupt: 